In [14]:
#####
## This version has two main edits
## 1) downsampled to much fewer frames
## 2) recurrent training 

import torch, argparse
import numpy as np
import pandas as pd
import time


import os, sys
THIS_DIR = os.path.dirname(os.path.abspath(""))
THIS_DIR = os.path.join(THIS_DIR, "GAT-HNN-R-v2")
# PARENT_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
# sys.path.append(PARENT_DIR)

from cuda_nn_models import *
from cuda_hnn import HNN
from cuda_utils import L2_loss, to_pickle, from_pickle
from get_data import elapsed
# from get_data import get_dataset ## we dont need this now
import glob


##
# Loading in the whitened Dataset
##
num_atoms = 40
batch_size = 100
epochs = 3
save_every = 500 ## how often the model is saved


start_time = time.time()
# print("Elapsed time: ", elapsed(time.time() - start_time))


PATH = "models/GATHNNRv1.pt"
raw_data = './../data/SMD_data/*.npy'
saved_x_dataset = "../data/ds_250_uw_x_dataset.npy"
saved_dx_dataset = "../data/ds_250_uw_dx_dataset.npy"
downsample_num = 80
log_csv_name = "log.csv"

data_whitened = False


### Determining current 
if not os.path.isfile(log_csv_name):
    print("The log file does not exist, creating log file...")
    log_df = pd.DataFrame(columns = ['time', 'epoch', 'loss'])
    current_epoch = 0
else:
    print("Found the log file! Reading it...")
    log_df = pd.read_csv(log_csv_name)
    current_epoch = log_df.iloc[-1]['epoch']
    print("Current Epoch is: ", current_epoch)
    
def add_log_csv(log_df, log_csv_name, start_time, epoch, loss):
    time_elapsed = elapsed(time.time() - start_time)
    next_step = pd.DataFrame([{'time': time_elapsed, 'epoch': epoch, 'loss': loss}])
    log_df = log_df.append(next_step)
    log_df.to_csv(log_csv_name, index=False)
    return log_df

The log file does not exist, creating log file...


In [ ]:
 
def get_dataset(raw_data, saved_x_dataset, saved_dx_dataset, num_atoms, downsample_num, data_whitened):
    
    if data_whitened == True:
        print("Preparing Whitened Data...")
        num_trajectories = 200
        
        files = glob.glob(raw_data)
        dataset = []
        for file_ in files:
            print("Loading File", file_)
            X_positions = np.load(file_)
            downsampled_X = X_positions[::downsample_num]
            dataset.append(downsampled_X)

        dataset = np.array(dataset)
        dataset = dataset.reshape(num_trajectories, -1, num_atoms*3)
    else:
        print("Preparing unwhitened Data...")
        num_trajectories = 100
        num_atoms = 40

        dataset = []
        for i in range(num_trajectories):
            NUM = str(i).zfill(2)
            fName = "../../../10_deca_alanine/" + NUM + "/backbone.npy"
            print("Loading file", fName)
            rawCoord = np.load(fName)
            downsampled_X = rawCoord[::downsample_num]
            dataset.append(downsampled_X)

        dataset = np.array(dataset) ## shape: (100, 20000, 40, 3)
        dataset = dataset.reshape(num_trajectories, -1, num_atoms*3)


    ### Preprocessing to create the training step

    x_dataset = [] ## vector of position and momentum
    dx_dataset = [] ## vector of change in position and momentum
    ## Getting the velocity of each trajectory
    
    for traj_idx in range(num_trajectories):
        traj = dataset[traj_idx]
    #     traj = dataset[0]
        num_datapoints = traj.shape[0] - 1

        momenta = []
        for i in range(traj.shape[0] - 1):
            dx = traj[i+1] - traj[i]
            momenta.append(dx)

        momenta = np.array(momenta) ## convert to an np array

        ## stack the coordinates so that we have all the positions and all the momentums
        coords = np.stack((traj[:-1], momenta), 1) ## shape: (19999, 2, 120)
        coords = coords.reshape(num_datapoints, 2 * 120) ## shape: (19999, 240)

        ## Now we need to calculate the change in coords (positions and momentums)
        delta_coords = []
        for frame in range(coords.shape[0] - 1):
            dx = coords[frame+1] - coords[frame]
            delta_coords.append(dx)
        delta_coords = np.array(delta_coords)

        ## appending to the dataset
        x_dataset.append(coords[:-1])
        dx_dataset.append(delta_coords)


    x_dataset = np.array(x_dataset) ## shape of (100, 19998, 240)
    dx_dataset = np.array(dx_dataset) ## shape of (100, 19998, 240)

#     x_dataset = x_dataset.reshape(-1, 240) ## shape of (1999800, 240)
#     dx_dataset = dx_dataset.reshape(-1, 240) ## shape of (1999800, 240)

    # x_dataset[0] + dx_dataset[0] == x_dataset[1] ## This is just a sanity check that we can predict the next step

    ## just for quick load don't need this
    np.save(saved_x_dataset, x_dataset)
    np.save(saved_dx_dataset, dx_dataset)
    del dataset
    return x_dataset, dx_dataset

x_dataset, dx_dataset = get_dataset(raw_data, saved_x_dataset, saved_dx_dataset, num_atoms, downsample_num, data_whitened)

Preparing unwhitened Data...
Loading file ../../../10_deca_alanine/00/backbone.npy
Loading file ../../../10_deca_alanine/01/backbone.npy
Loading file ../../../10_deca_alanine/02/backbone.npy
Loading file ../../../10_deca_alanine/03/backbone.npy
Loading file ../../../10_deca_alanine/04/backbone.npy
Loading file ../../../10_deca_alanine/05/backbone.npy
Loading file ../../../10_deca_alanine/06/backbone.npy
Loading file ../../../10_deca_alanine/07/backbone.npy
Loading file ../../../10_deca_alanine/08/backbone.npy
Loading file ../../../10_deca_alanine/09/backbone.npy
Loading file ../../../10_deca_alanine/10/backbone.npy
Loading file ../../../10_deca_alanine/11/backbone.npy
Loading file ../../../10_deca_alanine/12/backbone.npy
Loading file ../../../10_deca_alanine/13/backbone.npy
Loading file ../../../10_deca_alanine/14/backbone.npy
Loading file ../../../10_deca_alanine/15/backbone.npy
Loading file ../../../10_deca_alanine/16/backbone.npy
Loading file ../../../10_deca_alanine/17/backbone.npy

In [45]:
# x_dataset.shape ##(100, 248, 240)
## Now we have to process these into sequences of 10 or whatever you want
seq_len = 10

num_traj = x_dataset.shape[0]
num_points = x_dataset.shape[1]

# dataset = []
x_dat = []
dx_seq_dat = []
for traj in range(num_traj):
    x = x_dataset[traj]
    dx = dx_dataset[traj]
    
    for i in range(num_points - seq_len):
        x_dat.append(x[i])
        dx_seq_dat.append(dx[i:i+seq_len])
#         position = torch.tensor(x[i], requires_grad=True, dtype=torch.float32).cuda()
#         momenta_sequence = torch.Tensor(dx[i:i+seq_len]).cuda()
#         dataset.append((position,momenta_sequence ))

x_dat = np.array(x_dat)
dx_seq_dat = np.array(dx_seq_dat)

x_dat = torch.tensor(x_dat, requires_grad=True, dtype=torch.float32).cuda()
dx_seq_dat = torch.Tensor(dx_seq_dat).cuda()

print("current x_dataset size: ", x_dat.shape)
print("current dx_dataset size: ", dx_seq_dat.shape)

current x_dataset size:  torch.Size([23800, 240])
current dx_dataset size:  torch.Size([23800, 10, 240])


In [115]:
class GATModel(torch.nn.Module):

    def __init__(self,channel_size, hidden_size, output_size):
        super(GATModel, self).__init__()
        self.gat_encoder = GATEncoder(channel_size, hidden_size)
        self.gat_decoder = GATDecoder(hidden_size, output_size)
        self.gat_processor = GATProcessor(hidden_size, 3)
        self.transform0 = T.KNNGraph(k=4)
        self.transform = T.Distance(norm=True)
  
    def forward(self,x):
        x = x[:,120:].view(x.size()[0],40,3)
        r = x[:,:120].view(x.size()[0],40,3)
    
        outputs = []
        for i in range(x.size()[0]):
            xi = Data(x=x[i,:,:], pos=r[i,:,:])
            xi = self.transform0(xi)
            xi = self.transform(xi)

            ###
            # GAT Encoder
            ###
            x_encoded = self.gat_encoder(xi.x, xi.edge_index)

            ###
            # GAT Processor
            ###
            x_processed = self.gat_processor(x_encoded, xi.edge_index)

            ###
            # GAT Decoder
            ###
            x_decoded = self.gat_decoder(x_processed, xi.edge_index)

            # x output
            output = x_decoded.sum(dim=0)
#             print(x_decoded)
            outputs.append(output)
        outputs = torch.stack(outputs)
        return outputs

In [127]:
# batch_size = 5
# ixs = torch.randperm(x.shape[0])[:batch_size]
# a = x_dat[ixs]
# dxdt = dx_seq_dat[ixs]
# test = []
# for i in range(seq_len):
#     dxdt_hat = model.time_derivative(a)
#     a = a + dxdt_hat
#     test.append(dxdt_hat)

# dxdt_hat = torch.stack(test)
# dxdt_hat = torch.transpose(dxdt_hat, 0, 1)

In [139]:
## Now creating the model
#### LOADING THE PARAMETERS

# arrange data
def get_args():
    return {'input_dim': 240, # 40 atoms, each with q_x, q_y, p_z, p_y
         'hidden_dim': 200,
         'learn_rate': 1e-3,
         'input_noise': 0., ## NO INPUT NOISE YET
         'batch_size': 100,
         'nonlinearity': 'softplus',
#          'total_steps': total_steps, ## 3 epochs effectively i guess
         'field_type': 'helmholtz', ## change this? solenoidal
         'print_every': 5,
         'verbose': True,
         'name': '2body',
         'baseline' : False,
         'seed': 0,
         'save_dir': '{}'.format(THIS_DIR),
         'fig_dir': './figures'}


class ObjectView(object):
    def __init__(self, d): self.__dict__ = d

args = ObjectView(get_args())



#### THIS IS WHERE YOU INTERCEPT IF YOU WANT TO USE A DIFFERENT MODEL
output_dim = 2
# nn_model = MLP(args.input_dim, args.hidden_dim, output_dim, args.nonlinearity)
num_particles = 40
channel_size = 3
hidden_size = 64
output_size = 2
nn_model = GATModel(channel_size, hidden_size, output_size).cuda()

model = HNN(args.input_dim, differentiable_model=nn_model,
        field_type=args.field_type, baseline=args.baseline)
optim = torch.optim.Adam(model.parameters(), args.learn_rate, weight_decay=0)


In [140]:
# vanilla train loop
stats = {'train_loss': [], 'test_loss': []}

batch_size = 10


for epoch in range(epochs):
    for step in range(int(x_dat.shape[0] / batch_size)):
        
        
        ixs = torch.randperm(x.shape[0])[:batch_size]
        a = x_dat[ixs]
        dxdt = dx_seq_dat[ixs]
        traj_output = []
        for _ in range(seq_len):
            dxdt_hat = model.time_derivative(a)
            a = a + dxdt_hat
            traj_output.append(dxdt_hat)

        dxdt_hat = torch.stack(traj_output)
        dxdt_hat = torch.transpose(dxdt_hat, 0, 1)

        loss = L2_loss(dxdt, dxdt_hat)
        loss.backward()
        optim.step() ; optim.zero_grad()
        
#         print(step, " || ", loss.item())

        # logging
        stats['train_loss'].append(loss.item())
        if args.verbose and step % args.print_every == 0:
            print("step {}, train_loss {:.4e}"
              .format(step, loss.item()))
            
            f = open("trainlog.txt", "a")
            f.write("step {}, train_loss {:.4e}\n"
              .format(step, loss.item()))
            f.close()

        if step % save_every == 0 and step != 0:
            torch.save(model.state_dict(), PATH) ### SAVING THE MODEL EVERY FEW STEPS
            write_epoch = current_epoch + epoch ## this gets updated everytime the file terminates
            add_log_csv(log_df, log_csv_name, start_time, write_epoch, loss.item())

print("=> Finished Training <=")


## Saving model
torch.save(model.state_dict(), PATH) ### SAVING THE MODEL EVERY FEW STEPS
write_epoch = current_epoch + epoch ## this gets updated everytime the file terminates
add_log_csv(log_df, log_csv_name, start_time, write_epoch, loss.item())


step 0, train_loss 1.4275e+00
step 5, train_loss 1.4317e+00


KeyboardInterrupt: 